## Cost Function

In [65]:
def cost_function(params):
    # Unpack parameters
    Nx, Ny, Lx, Ly, h, Nr, Lr, A_grid, t_rev = params

    # Material costs (per unit)
    c_conductor = 0  # $/m
    c_excavation = 261.59 # $/m³
    c_welding = 12  # $/joint
    c_rod = 0  # $/m
    c_rev = 25  # $/m³

    # Cost components
    C1 = (Nx * Lx + Ny * Ly) * c_conductor
    C2 = (Nx * Lx + Ny * Ly) * h * c_excavation
    C3 = (Nx + Ny) * c_welding
    C4 = Nr * Lr * c_rod
    C5 = A_grid * t_rev * c_rev
    C6 = A_grid * h * c_excavation

    # Total cost
    C_total = C1 + C2 + C3 + C4 + C5
    return C_total

# DE Method

In [58]:
import numpy as np

def differential_evolution(cost_func, bounds, pop_size=20, mutation=0.8, crossover=0.7, generations=200):
    dimensions = len(bounds)
    pop = np.random.rand(pop_size, dimensions)
    min_b, max_b = np.asarray(bounds).T
    diff = np.fabs(min_b - max_b)
    pop_denorm = min_b + pop * diff
    fitness = np.asarray([cost_func(ind) for ind in pop_denorm])
    best_idx = np.argmin(fitness)
    best = pop_denorm[best_idx]
    
    for i in range(generations):
        for j in range(pop_size):
            idxs = [idx for idx in range(pop_size) if idx != j]
            a, b, c = pop[np.random.choice(idxs, 3, replace=False)]
            mutant = np.clip(a + mutation * (b - c), 0, 1)
            cross_points = np.random.rand(dimensions) < crossover
            if not np.any(cross_points):
                cross_points[np.random.randint(0, dimensions)] = True
            trial = np.where(cross_points, mutant, pop[j])
            trial_denorm = min_b + trial * diff
            f = cost_func(trial_denorm)
            if f < fitness[j]:
                fitness[j] = f
                pop[j] = trial
                if f < fitness[best_idx]:
                    best_idx = j
                    best = trial_denorm
    return best, fitness[best_idx]


In [59]:
def differential_evolution_for_max(cost_func, bounds, pop_size=20, mutation=0.8, crossover=0.7, generations=200):
    dimensions = len(bounds)
    pop = np.random.rand(pop_size, dimensions)
    min_b, max_b = np.asarray(bounds).T
    diff = np.fabs(min_b - max_b)
    pop_denorm = min_b + pop * diff
    fitness = np.asarray([cost_func(ind) for ind in pop_denorm])
    best_idx = np.argmax(fitness)
    best = pop_denorm[best_idx]
    
    for i in range(generations):
        for j in range(pop_size):
            idxs = [idx for idx in range(pop_size) if idx != j]
            a, b, c = pop[np.random.choice(idxs, 3, replace=False)]
            mutant = np.clip(a + mutation * (b - c), 0, 1)
            cross_points = np.random.rand(dimensions) < crossover
            if not np.any(cross_points):
                cross_points[np.random.randint(0, dimensions)] = True
            trial = np.where(cross_points, mutant, pop[j])
            trial_denorm = min_b + trial * diff
            f = cost_func(trial_denorm)
            if f > fitness[j]:
                fitness[j] = f
                pop[j] = trial
                if f > fitness[best_idx]:
                    best_idx = j
                    best = trial_denorm
    return best, fitness[best_idx]

In [61]:
bounds = [
    (58, 134),  # conductors per direction over a 400 m span
    (58, 134),  # conductors per direction over a 400 m span
    (400, 400),    # Lx: exact grid side length in X (m)
    (400, 400),    # Ly: exact grid side length in Y (m)
    (0.5, 1.0),          # h: burial depth (m) per IEEE 80, 0.3–0.5 m typical, 0.5–1 m for 345 kV :contentReference[oaicite:2]{index=2}
    (100, 200),          # Nr: number of rods (≈ perimeter/15 m + equipment ties) :contentReference[oaicite:3]{index=3}
    (2.44, 7.5),         # Lr: rod length (m), 8 ft (2.44 m) standard to 7.5 m for deep soils :contentReference[oaicite:4]{index=4}
    (161874.0, 161874.0),# A_grid: fixed yard area (m²) = 40 ac × 4046.856 m²/ac
    (0.10, 0.20)         # t_rev: revetment thickness (m), ~6 in (0.15 m) per IEEE 80 :contentReference[oaicite:5]{index=5}
]


In [62]:
best_params, min_cost = differential_evolution(cost_function, bounds)
print("Optimal Parameters:", best_params)
print(f"Minimum Cost: ${min_cost:,.2f}")

Optimal Parameters: [5.80000000e+01 5.80000000e+01 4.00000000e+02 4.00000000e+02
 5.00000000e-01 1.93961537e+02 4.19906752e+00 1.61874000e+05
 1.00000000e-01]
Minimum Cost: $6,474,965.00


In [63]:
best_params, max_cost = differential_evolution_for_max(cost_function, bounds)
print("Optimal Parameters:", best_params)
print(f"Maximum Cost: ${max_cost:,.2f}")

Optimal Parameters: [1.34000000e+02 1.34000000e+02 4.00000000e+02 4.00000000e+02
 1.00000000e+00 1.00000000e+02 2.78682548e+00 1.61874000e+05
 2.00000000e-01]
Maximum Cost: $28,855,034.00


# SCA Method

In [64]:
import numpy as np

def sine_cosine_optimizer_ext(cost_func, cost_params, bounds,
                              pop_size=30, a=2.0, max_iter=200):
    """
    Sine–Cosine Algorithm that returns both the minimum-cost and maximum-cost solutions.
    """

    dim = len(bounds)
    min_b, max_b = np.asarray(bounds).T
    diff = max_b - min_b

    # Initialize population
    pop = min_b + np.random.rand(pop_size, dim) * diff

    # Evaluate initial fitness
    fitness = np.array([cost_func(ind) for ind in pop])

    # Track best (min) and worst (max)
    best_idx = np.argmin(fitness)
    worst_idx = np.argmax(fitness)
    best = pop[best_idx].copy()
    worst = pop[worst_idx].copy()
    best_cost = fitness[best_idx]
    worst_cost = fitness[worst_idx]

    for t in range(1, max_iter + 1):
        r1 = a - t * (a / max_iter)  # linearly decreasing

        for i in range(pop_size):
            for j in range(dim):
                r2 = 2 * np.pi * np.random.rand()
                r3 = 2 * np.random.rand()
                r4 = np.random.rand()

                if r4 < 0.5:
                    pop[i,j] += r1 * np.sin(r2) * abs(r3 * best[j] - pop[i,j])
                else:
                    pop[i,j] += r1 * np.cos(r2) * abs(r3 * best[j] - pop[i,j])

            # Ensure bounds
            pop[i] = np.clip(pop[i], min_b, max_b)

            # Evaluate
            f = cost_func(pop[i])

            # Update best (minimize)
            if f < best_cost:
                best_cost = f
                best = pop[i].copy()

            # Update worst (maximize)
            if f > worst_cost:
                worst_cost = f
                worst = pop[i].copy()

    return (best, best_cost), (worst, worst_cost)


# -- Example Usage --

if __name__ == "__main__":
    # cost_params and bounds as before...
    cost_params = (0.0, 261.59, 12.0, 0.0, 25.0)
    bounds = [
        (58, 134), (58, 134),
        (400.0, 400.0), (400.0, 400.0),
        (0.5, 1.0), (100, 200),
        (2.44, 7.5), (161874, 161874),
        (0.10, 0.20)
    ]

    (best_design, best_cost), (worst_design, worst_cost) = sine_cosine_optimizer_ext(
        cost_function, cost_params, bounds, pop_size=50, a=2.5, max_iter=300
    )

    print("Lowest-cost design:", best_design, f"Cost = ${best_cost:,.2f}")
    print("Highest-cost design:", worst_design, f"Cost = ${worst_cost:,.2f}")


Lowest-cost design: [5.80000e+01 5.80000e+01 4.00000e+02 4.00000e+02 5.00000e-01 2.00000e+02
 7.50000e+00 1.61874e+05 1.00000e-01] Cost = $6,474,965.00
Highest-cost design: [1.34000000e+02 1.34000000e+02 4.00000000e+02 4.00000000e+02
 1.00000000e+00 1.60706192e+02 2.44000000e+00 1.61874000e+05
 2.00000000e-01] Cost = $28,855,034.00


# DE-SCA Method

In [67]:
import numpy as np


def hybrid_de_sca_optimizer(cost_func, bounds,
                            pop_size=30, F=0.8, CR=0.7, a=2.0, max_iter=200):
    """
    Hybrid DE–SCA optimizer returning both min and max cost solutions.
    Args:
      cost_func: function(params) -> cost
      bounds: list of (min, max) for each variable
      pop_size: population size
      F: DE mutation factor
      CR: DE crossover rate
      a: SCA exploration factor
      max_iter: total iterations

    Returns:
      (best_params, best_cost), (worst_params, worst_cost)
    """
    dim = len(bounds)
    min_b, max_b = np.asarray(bounds).T
    diff = max_b - min_b

    # Initialize population in original scale
    pop = min_b + np.random.rand(pop_size, dim) * diff
    fitness = np.array([cost_func(ind) for ind in pop])

    # Track best (minimization) and worst (maximization)
    best_idx = np.argmin(fitness)
    worst_idx = np.argmax(fitness)
    best = pop[best_idx].copy()
    worst = pop[worst_idx].copy()
    best_cost = fitness[best_idx]
    worst_cost = fitness[worst_idx]

    for t in range(1, max_iter + 1):
        # Differential Evolution step
        for i in range(pop_size):
            idxs = [idx for idx in range(pop_size) if idx != i]
            a_vec, b_vec, c_vec = pop[np.random.choice(idxs, 3, replace=False)]
            mutant = np.clip(a_vec + F * (b_vec - c_vec), 0, 1)
            cross = np.random.rand(dim) < CR
            if not np.any(cross):
                cross[np.random.randint(0, dim)] = True
            trial = np.where(cross, mutant, pop[i])
            trial_denorm = min_b + trial * diff
            f_trial = cost_func(trial_denorm)
            # Update population, best, worst
            if f_trial < fitness[i]:
                pop[i] = trial
                fitness[i] = f_trial
            if f_trial < best_cost:
                best_cost = f_trial
                best = trial_denorm.copy()
            if f_trial > worst_cost:
                worst_cost = f_trial
                worst = trial_denorm.copy()

        # Sine–Cosine Algorithm step
        r1 = a - t * (a / max_iter)
        for i in range(pop_size):
            for j in range(dim):
                r2 = 2 * np.pi * np.random.rand()
                r3 = 2 * np.random.rand()
                r4 = np.random.rand()
                if r4 < 0.5:
                    pop[i, j] += r1 * np.sin(r2) * abs(r3 * best[j] - pop[i, j])
                else:
                    pop[i, j] += r1 * np.cos(r2) * abs(r3 * best[j] - pop[i, j])
            pop[i] = np.clip(pop[i], 0, 1)
            denorm = min_b + pop[i] * diff
            f_val = cost_func(denorm)
            # Update best, worst
            if f_val < best_cost:
                best_cost = f_val
                best = denorm.copy()
            if f_val > worst_cost:
                worst_cost = f_val
                worst = denorm.copy()

    return (best, best_cost), (worst, worst_cost)


# Example usage
if __name__ == "__main__":

    bounds = [
        (58, 134),
        (58, 134),
        (400.0, 400.0),
        (400.0, 400.0),
        (0.5, 1.0),
        (100, 200),
        (2.44, 7.5),
        (161874, 161874),
        (0.10, 0.20)
    ]

    (best_params, best_cost), (worst_params, worst_cost) = hybrid_de_sca_optimizer(
        cost_function, bounds,
        pop_size=50, F=0.8, CR=0.7, a=2.0, max_iter=300
    )

    print("Optimal parameters:", best_params)
    print(f"Minimum grounding grid cost: ${best_cost:,.2f}")
    print("Worst parameters:", worst_params)
    print(f"Maximum grounding grid cost: ${worst_cost:,.2f}")

Optimal parameters: [5.80000000e+01 5.80000000e+01 4.00000000e+02 4.00000000e+02
 5.00000000e-01 1.00000000e+02 4.35842419e+00 1.61874000e+05
 1.00000000e-01]
Minimum grounding grid cost: $6,474,965.00
Worst parameters: [8.24152608e+03 8.72885069e+03 4.00000000e+02 4.00000000e+02
 1.00000000e+00 2.00000000e+02 2.46396247e+01 1.61874000e+05
 1.13186463e-01]
Maximum grounding grid cost: $1,776,374,037.12
